In [1]:
#Step 1. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다
from bs4 import BeautifulSoup     
from selenium import webdriver
import time
import sys

In [2]:
print("fatsecret 사이트에서 제품의 영양관련 정보를 'xlsx' 파일로 긁어오는 크롤러입니다.")
print("경로 지정을 할 때 반드시 xlsx 확장자로 지정하세요.")
f_name = input('검색 결과를 저장할 파일경로와 이름을 지정하세요(예:c:\\data\\test.xlsx): ')


#Step 2. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
path = "c:/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("http://yangsan.heemang.or.kr/heemang/merchant/mgr/initMerchantList.han")
time.sleep(2)  #  창이 모두 열릴 때 까지 2초 기다립니다.

#Step 3. 검색창의 이름을 찾아서 검색어를 입력하고 검색을 실행합니다
driver.find_element_by_xpath('//*[@id="content"]/div[2]/div/div/input[2]').click()

time.sleep(2) # 창이 켜질때까지 2초 대기합니다.

full_html = driver.page_source #현재 페이지의 html을 긁어온다
soup = BeautifulSoup(full_html, 'html.parser') #파서의 이름을 soup로 지정.

MarketNum = soup.select('#content > div.contents > div > table > tbody:nth-child(5) > tr > td')

MarketNum = MarketNum[0].text.strip().replace('전체 ','').replace(' 건','')

import math
PageNum = math.ceil(int(MarketNum) / 10) #페이지 갯수

print("총 가게는 %s개이며 수집할 페이지는 %s입니다." %(MarketNum,PageNum))

TotalMarketName = []
TotalPlace = []
TotalCall = []

no = 1
for i in range(PageNum) :
    full_html = driver.page_source #현재 페이지의 html을 긁어온다
    soup = BeautifulSoup(full_html, 'html.parser') #파서의 이름을 soup로 지정.
    Writer = soup.find_all('td',class_='writer') #가게 명 및 위치
    Dat = soup.find_all('td', class_='dat') # 가게 전화번호
    for j in range(len(Writer)):
        if j % 2 == 0:
            TotalMarketName.append(Writer[j].text.strip())
        else:
            TotalPlace.append(Writer[j].text.strip())
    no += 1
     
    for k in range(len(Dat)):
        TotalCall.append(Dat[k].text.strip())
        
    if i+2 <= PageNum:
        driver.find_element_by_xpath('//*[@id="bottom_m"]/ul/li[4]/a').location_once_scrolled_into_view # 원활한 클릭을 위해 스크롤 내리기
        time.sleep(0.5)
        if no % 11 == 0 :
            no -= 10
            time.sleep(0.5)
            driver.find_element_by_link_text("다음").click()
        else:
            driver.find_element_by_link_text(str(i+2)).click()
            time.sleep(0.5)
    else :
        break
    

import pandas as pd

store = pd.DataFrame()
store['가맹점명'] = TotalMarketName
store['주소'] =  TotalPlace
store['전화번호'] =  TotalCall

store.to_excel(f_name, sheet_name='sheet1')

fatsecret 사이트에서 제품의 영양관련 정보를 'xlsx' 파일로 긁어오는 크롤러입니다.
경로 지정을 할 때 반드시 xlsx 확장자로 지정하세요.


검색 결과를 저장할 파일경로와 이름을 지정하세요(예:c:\data\test.xlsx):  C:/Users/zxc80/Desktop/빅리더_꿈나무/양산시가맹점.xlsx


총 가게는 351개이며 수집할 페이지는 36입니다.


In [ ]:
s